<a href="https://colab.research.google.com/github/Udhayakumarbalasubramaniam/PRODIGY/blob/main/PRODIGY_Trackcode_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'customer-segmentation-tutorial-in-python:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F42674%2F74935%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240603%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240603T004035Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3a80ea2f930bb276219e1c5cfe558b2b572ff2dd4d7f66dbe940b274be329677e756f8434ff8e471e02e4c4a6ad9e6eed88e97d509c89e94ea278fd7136f7aace18966bcb51a24db08ce0019c3535f0dfa7637e8fad714ffd7bad5f6222a82b71445a70170aee01f1c29a27253d6487488d14720c7d1621e67b89b086df04487ff3aafd93ed5ab09c0698adb3913bd9f5dc9e2f19e746bfc7d72112840eb487e60575c836278374a656a177f43f9b81cfafa58b17e8272c6fa0c84f3cc0ac46a6f53fc1d263354a0b185f9e1d127795b223df4d12a2edfe11fc577e16d1b2aad39da1291c9951d71d6108d36362056df977ce9ad212b8d9cddbeec0c0e9a656d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# This notebook is for the practice purpose for my data science skill

Here, all about the needed basic skill of EDA and basic modeling along with visualization along various interactive libraries like plotly, altair, seaborn and so on will be done


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Import all necessary packages

In [ ]:
#Import packages
import pandas as pd
import numpy as np
from collections import Counter

#For visualization
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


#For other models
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from kmodes.kmodes import KModes
from kmodes import kprototypes

#Calculate distance matrix to cluster mixed data type
from scipy.spatial import distance_matrix

#For calculate solhouette_score
from sklearn.metrics import silhouette_score

#For Encoding
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


# For metrics and helpful functions
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

#warning
import warnings

In [ ]:
warnings.filterwarnings("ignore") # To prevent kernel from showing any warning

# 2. Load Dataset

In [ ]:
#Load dataset
df = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
#Display first 10 rows of dataset
df.head(10)

In [ ]:
#Rename columns
df = df.rename(columns={"Annual Income (k$)": "Annual Income", "Spending Score (1-100)": "Spending Score"})
#Display bottom 10 rows
df.tail(10)

# 3. EDA (Exploratory Data Analysis)

## 3.1 Basic Info

In [ ]:
#Data shape
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns in the data')

In [ ]:
#dataset info
df.info()

**Pandas**  is a popular heterogeneous data managing library that hold data in 2 form either in Series or in data frame, heere data is in data frame,it is very much used while manipulation of heterogenous data.

## 3.2 Check for missing and duplicate values

In [ ]:
#Check for missing value
df.isna().sum()

In [ ]:
#Check for duplicates
df.duplicated().sum()

## 3.3 Drop unecessary columns

In [ ]:
#Drop `CustomerID` that not need for modeling
clean_df = df.drop(['CustomerID'],axis=1)

clean_df.head(7)

## 3.4 Descriptive statistics about the data

In [ ]:
#Display basic descriptive stats
clean_df.describe()

This provide a very general overview of our data that help in the initial phase of data analysis.

## 3.5 Check & Handle outliers

In [ ]:
#Create subplot to visualize outliers
fig = px.box(clean_df, y="Annual Income")
fig.show()

**Plotly** is an interactive visualizing app which is used mostly after one get good at visulization knowledge initiated mostly through matplotlib.

This function could be used for the calculation of outliers in the dataset, since we don't have much in our data, which is clearly visible in the box plot.

In [ ]:
#Create function to calculate the upper limit, lower limit, number of outliers and their % in dataset for each specified column
def calculate_outliers(df, column_names):
    """
    Calculate upper limit, lower limit, number of outliers and the % in dataset for each specified column .

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data and need to calculate outliers
    - column_names (list): A list of column names to analyze for outliers.

    Returns:
    - pd.DataFrame: A DataFrame containing information about outliers for each specified column.
      Columns: ['Column', 'Number of Outliers', '% Outliers', 'Lower Limit', 'Upper Limit']
    """

    # Calculate the number of columns and rows for subplots
    col_len = len(column_names)
    if col_len % 2 == 0:
      num_columns = 2
      num_rows = col_len // 2
    else:
      num_columns = col_len
      num_rows = 1


    # Initialize a list to store outlier information for each column
    outlier_data = []

    # Iterate over column_names and create subplots
    for i, column_name in enumerate(column_names):
        data = df[column_name]

        # Calculate quartiles and interquartile range using np.percentile
        q1, q3 = np.percentile(data, [25, 75])
        iqr = q3 - q1

        # Define lower and upper limits for outlier detection
        low_lim, upp_lim = q1 - 1.5 * iqr, q3 + 1.5 * iqr

        # Find and display outliers
        outliers = df[(data < low_lim) | (data > upp_lim)][column_name]
        num_outliers = len(outliers)
        percent_outliers = round(num_outliers / len(df) * 100, 1)

        # Append outlier information to the list
        outlier_data.append([column_name, num_outliers, percent_outliers, round(low_lim, 1), round(upp_lim, 1)])


    # Combine the outlier information into a DataFrame
    outlier_df = pd.DataFrame(outlier_data,
                              columns = ['Column', 'Number of Outliers', '% Outliers', 'Lower Limit', 'Upper Limit'])
    return outlier_df

In [ ]:
#Define column to check
col=['Annual Income']
#use function `calculate_outliers` to detect and visualize outliers in table
outliers_table = calculate_outliers(clean_df,col)
#Display outliers table
outliers_table

In [ ]:
clean_df['Gender'].value_counts()

In [ ]:
sns.displot(clean_df, x = 'Age',hue='Gender', kind='kde')

In [ ]:
#Visulize `Gender` distribution

##Create series to count values in `Gender` column
gender_dis = clean_df['Gender'].value_counts()

##visualize in pie chart
fig = px.pie(
             names=gender_dis.index,
             values= gender_dis.values,
             title='<b>Gender Count</b>',
             hole=.4,
             width=400,
             height=400
             )
fig.update_traces(textposition='inside',
                  textinfo='percent+label+value',
                  textfont_size=15,
                  marker=dict(colors=['#BB2525', '#279EFF'], line=dict(color='#000000', width=2)))
fig.update_layout(title={'font': {'size': 25}}, showlegend=False)
fig.show()

In [ ]:
avg_cost = clean_df['Age'].mean().round(0)

fig = px.histogram(
    clean_df,
    x = 'Age',
    color='Gender',
    # labels={'comb08': "Annual Fuel Cost"},
    nbins=10,
    title='Age Distribution',
    color_discrete_sequence=px.colors.sequential.Magenta_r,
)
fig.add_vline(
    x = avg_cost,
    annotation_text = f'Average {avg_cost}'
)
fig.show()

In [ ]:
clean_df.tail()

In [ ]:
avg_cost = clean_df['Annual Income'].mean().round(0)

fig = px.histogram(
    clean_df,
    x = 'Annual Income',
    color='Gender',
    # labels={'comb08': "Annual Fuel Cost"},
    nbins=10,
    title='Annual Income Distribution',
    color_discrete_sequence=px.colors.sequential.Magenta_r,
)
fig.add_vline(
    x = avg_cost,
    annotation_text = f'Average ${avg_cost}k'
)
fig.show()

In [ ]:
df['Annual Income'].max()

In [ ]:
avg_cost = clean_df['Spending Score'].mean().round(0)

fig = px.histogram(
    clean_df,
    x = 'Spending Score',
    color='Gender',
    # labels={'comb08': "Annual Fuel Cost"},
    nbins=10,
    title='Spending Score Distribution',
    color_discrete_sequence=px.colors.sequential.Magenta_r,
)
fig.add_vline(
    x = avg_cost,
    annotation_text = f'Average ${avg_cost}k'
)
fig.show()

Based on the data presented in the charts, we can make several key observations:

- The distribution of Annual Income appears to be somewhat normal, with a majority of customers falling within the  $60,000 -  $80,000 range.
- The mall customers majority are under 40 years old.
- The Spending Score also follows a normal distribution pattern.

In [ ]:
fig = px.bar(
    clean_df,
    x = 'Age',
    y = 'Annual Income',
    color='Gender',
)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = px.bar(
    clean_df,
    x='Age',
    y='Annual Income',
    color = 'Gender'
    )

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = px.bar(
    clean_df,
    x='Age',
    y='Spending Score',
    color = 'Gender'
    )

fig.show()

In [ ]:
fig = px.scatter(
    clean_df,
    x = 'Age',
    y = 'Annual Income',
    trendline='ols'
)
fig.show()

In [ ]:
fig = px.scatter(
    clean_df,
    x = 'Age',
    y = 'Annual Income',
    trendline='lowess'
)
fig.show()

In [ ]:
clean_df.tail(7)

In [ ]:
fig = px.sunburst(
    clean_df,
    path = ['Gender', 'Age', 'Annual Income'],
    values='Spending Score'
)

fig.show()

In [ ]:
sns.pairplot(clean_df,hue='Gender')

In [ ]:
#Create heatmap to check correlation of all numerical features
plt.figure(figsize=(10,8))
sns.heatmap(clean_df.drop('Gender',axis=1).corr(), annot=True, cmap=plt.cm.Reds)

- There is an inverse relationship between Age and both Annual Income and Spending Score, suggesting that as age increases, annual income and spending score tend to decrease.
- Annual Income and Spending Score exhibit a positive correlation, meaning that as annual income increases, spending score also tends to increase.
- Overall, there is no strong correlation observed among the three features, Age, Annual Income, and Spending Score

# 4. Modeling

In [ ]:
#Define number of cluster want to fit and evaluate
num_clusters = [i for i in range(2, 16)]
num_clusters

In [ ]:
#Create subset dataframe with 2 features: `Annual Income` and `Spending Score`
X_2dim = clean_df[['Annual Income','Spending Score']]

In [ ]:
#Standarize the data
X_2dim_scaled = StandardScaler().fit_transform(X_2dim)
#Transform to dataframe
X_2dim_scaled = pd.DataFrame(X_2dim_scaled,columns = X_2dim.columns)
#Double check dataframe
X_2dim_scaled.describe()

In [ ]:
#Funtion to fit kmeans and evaluate inertia for different number of cluster

def kmeans_inertia(num_clusters, x_vals):
  """
  Accepts as arguments list of ints and data array.
  Fits a KMeans model where k = each value in the list of ints.

  Parameters:
    - num_clusters: number of cluster will fit and evaluate inertia
    - x_vals: the dataframe need to fit and evaluate inertia

  Returns list of inertia value for each k using KMeans model to fit.
  """
  inertia = []
  for num in num_clusters:
    kms = KMeans(n_clusters=num, random_state=20)
    kms.fit(x_vals)
    inertia.append(kms.inertia_)

  return inertia

In [ ]:
#Call kmeans_inertia function to return list of inertia
inertia_2dim = kmeans_inertia(num_clusters,X_2dim)
#Display list of inertia
inertia_2dim

In [ ]:
#Create line plot to visualize inertia and number of cluster respectively
plot = sns.lineplot(x=num_clusters, y=inertia_2dim, marker = 'o')
plot.set_xlabel("Number of clusters");
plot.set_ylabel("Inertia");

As expected, we can see an 'elbow' clearly at 5 clusters. Next step, we will calculate silhouette score for each cluster in range of num_clusters

In [ ]:
#Fit a 5-clusters model
kmeans5_2dim = KMeans(n_clusters=5, n_init='auto', random_state=50)
kmeans5_2dim.fit(X_2dim_scaled)

In [ ]:
#Use Counter to count number of data points in each cluster
Counter(kmeans5_2dim.labels_)

In [ ]:
#Create heatmap to visulize cluster center of each cluster
clus_cen5_2dim = pd.DataFrame(kmeans5_2dim.cluster_centers_, columns=X_2dim_scaled.columns)
sns.heatmap(clus_cen5_2dim, cmap='RdBu', annot=True);

As obseration, below are characteristics of 5 clusters:

- Cluster 0: High income but low spending score
- Cluster 1: Average income and average spending score
- Cluster 2: Low income and high spending score
- Cluster 3: High income and high spending score
- Cluster 4: Low income and low spending score

In [ ]:
kmeans5_2dim.labels_

In [ ]:
clean_df['cluster'] = kmeans5_2dim.labels_

In [ ]:
#Create scatterplot to visualize clusters in 2 dimensions
sns.scatterplot(x='Annual Income',
                y='Spending Score',
                hue=kmeans5_2dim.labels_,
                data=clean_df,
                palette='viridis');

In [ ]:

fig = px.scatter(clean_df, x="Annual Income", y="Spending Score", color="cluster", symbol="cluster")
fig.show()

In [ ]:
fig = px.scatter(clean_df, x="Annual Income", y="Spending Score", color="cluster",
                 size='Spending Score', hover_data=['Spending Score'])
fig.show()

In [ ]:
clean_df.tail()

In [ ]:
#Create dataframe with 2 features: `Annual Income` and `Spending Score`
X_3dim = clean_df.drop('Gender',axis=1)
#Double check dataframe
X_3dim.info()

In [ ]:
#Create dataframe with 2 features: `Annual Income` and `Spending Score`
X_3dim = X_3dim.drop('cluster',axis=1)
#Double check dataframe
X_3dim.info()

In [ ]:
#Standarize the dataframe
X_3dim_scaled = StandardScaler().fit_transform(X_3dim)
#Transform to dataframe
X_3dim_scaled = pd.DataFrame(X_3dim_scaled,columns = X_3dim.columns)
#Double check dataframe
X_3dim_scaled.describe()

In [ ]:
##Call kmeans_inertia function to return list of inertia for num_clusters
inertia_3dim = kmeans_inertia(num_clusters, X_3dim_scaled)
inertia_3dim

In [ ]:
#Create a line plot to visualize internia for each cluster
plot = sns.lineplot(x=num_clusters, y=inertia_3dim, marker = 'o')
plot.set_xlabel("Number of clusters");
plot.set_ylabel("Inertia");

In [ ]:
#Fit a 6-cluster model
kmeans6_3dim = KMeans(n_clusters=6,n_init='auto',random_state=50)
kmeans6_3dim.fit(X_3dim_scaled)

In [ ]:
#Find the number of data points in each cluster using Counter
Counter(kmeans6_3dim.labels_)

In [ ]:
kmeans6_3dim.labels_

In [ ]:
#Create a heat map of the cluster centers
plt.figure(figsize=(10,8))
clus_cen6_3dim = pd.DataFrame(kmeans6_3dim.cluster_centers_, columns=X_3dim_scaled.columns)
sns.heatmap(clus_cen6_3dim, cmap='RdBu', annot=True);

In [ ]:
#Add cluster label to dataframe
clean_df['kmeans6_3dim_labels'] = kmeans6_3dim.labels_
#Change type to category
clean_df['kmeans6_3dim_labels'] = clean_df['kmeans6_3dim_labels'].astype('category')
#Create boxplot visual to see compare each cluster in each feature


In [ ]:


#3D scatterplot to show all clusters
fig = px.scatter_3d(clean_df,
                    x='Annual Income',
                    y='Spending Score',
                    z='Age',
                    color='kmeans6_3dim_labels',
                    labels={"kmeans6_3dim_labels": "Cluster"},
                    category_orders={'kmeans6_3dim_labels':[0,1,2,3,4,5]})
fig.show()


Here we complete our basic Market Segmentation Basic Project that give a good overview of the system.